
# Entrenamiento del Modelo GPT-2 

El objetivo de este notebook es llevar acabo el *fine-tuning* del modelo GPT-2 de generación de texto en español, para que se desempeñe en la generación de texto para memes. El modelo GPT-2 se encuentra en almacenado en [la página de Hugging Face](https://huggingface.co/datificate/gpt2-small-spanish). 


Para ello es necesario descargar las librerías presentes en el archivo ``requirements.txt``

A continuación especificamos el contenido del notebook.

1. Instalación de librerías
2. Carga de Información
3. Preparación de Texto
4. Separación en Entrenamiento y Prueba
5. Entrenamiento del Modelo
6. Ejemplos de Texto Generado




# Instalación de Librerías

In [1]:
!cd "drive/MyDrive/Retrain" && git clone https://github.com/juancop/AIMemeGenerator # Se clona el repo porque se trabaja desde Google Colab

/bin/bash: line 0: cd: drive/MyDrive/Retrain: No such file or directory


In [ ]:
!cd "drive/MyDrive/Retrain/AIMemeGenerator" && pip install -r requirements.txt 

     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 2.1MB 42.9MB/s 
     |████████████████████████████████| 9.9MB 50.6MB/s 
     |████████████████████████████████| 22.3MB 1.4MB/s 
     |████████████████████████████████| 460kB 53.2MB/s 
     |████████████████████████████████| 901kB 52.0MB/s 
     |████████████████████████████████| 3.3MB 43.7MB/s 
ERROR: tensorflow 2.3.1 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
  Found existing installation: pandas 1.1.5
    Uninstalling pandas

La librería de transformers debe ser instalada directamente desde el repositorio

In [ ]:
! cd "/content/drive/MyDrive/Retrain/AIMemeGenerator/transformers" && git clone https://github.com/huggingface/transformers 

remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 82 (delta 56), reused 64 (delta 41), pack-reused 0
Unpacking objects: 100% (82/82), done.
From https://github.com/huggingface/transformers
   9046ff261..9f6743974  multi_regress     -> origin/multi_regress
   ed2331e68..38e841b84  new_training_tuto -> origin/new_training_tuto
Already up to date.


In [ ]:
!pip install -U pandas-profiling

     |████████████████████████████████| 245kB 7.2MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 1.1MB 11.6MB/s 
     |████████████████████████████████| 3.2MB 28.0MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 296kB 58.2MB/s 
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107413 sha256=084d17c27d8b78012d3b1f8a4ace6aa81926c146c42b96d9bdfa1819789e4070
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d85653789e80e966c3bf6
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27085 sha256=6fb01c4ae0a354c9c0a79cb35f0e0a190ded1867730ba86fc02b278bac851fbe
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
Successfully built phik htmlmin
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python

In [ ]:
!pip install datasets

     |████████████████████████████████| 225kB 8.5MB/s 
     |████████████████████████████████| 245kB 14.5MB/s 
     |████████████████████████████████| 112kB 16.8MB/s 


In [ ]:
! pip install git+git://github.com/huggingface/transformers/

  Cloning git://github.com/huggingface/transformers/ to /tmp/pip-req-build-gkkdmosi
  Running command git clone -q git://github.com/huggingface/transformers/ /tmp/pip-req-build-gkkdmosi
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2168795 sha256=df97380b4a7a33c60c013730a0fe250e4133a16e2b866636415c4a71de8b24a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-fc_62p5e/wheels/dc/e5/1e/3a2977a646558fca07585cadcf56aa4a910e995ba945961c4e
Successfully built transformers
  Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1


# 2. Carga de Información
En esta etapa cargaremos la información necesaria para el entrenamiento del modelo. En particular, se requiere la base de datos limpia y con filtros de groserías, ``base_final.csv``. Los textos de esta base de datos no contienen groserías, y serán limpiados para eliminar todos los caracteres que no sean alfanuméricos.


In [ ]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

In [ ]:
drive_path = 'drive/MyDrive/Retrain/AIMemeGenerator'

In [ ]:
df = pd.read_csv(os.path.join(drive_path, 'data/base_final/base_final.csv'))

In [ ]:
df.columns

Index(['id_imagen', 'meme_id', 'plantilla', 'texto_no_tilde', 'text_1',
       'text_2'],
      dtype='object')

In [ ]:
df = df[(~df.text_1.isna()) & (~df.text_2.isna())]
df["text_1"] = df["text_1"].str.lower()
df["text_2"] = df["text_2"].str.lower()
df['meme_id'] = df['meme_id'].apply('{:0>4}'.format)

In [ ]:
df = df[~(df.text_1.str.contains('\|') & df.text_2.str.contains('\|'))]
len(df)

676777

In [ ]:
df['text_1'] = df.text_1.str.replace(r'[^A-Za-z0-9 ?¿!¡]+', r'')
df['text_2'] = df.text_2.str.replace(r'[^A-Za-z0-9 ?¿!¡]+', r'')

df["text_1"] = df["text_1"].str.replace('\s+', ' ', regex=True)
df['text_2'] = df["text_2"].str.replace('\s+', ' ', regex=True)

# 3. Preparación de Texto

Para el entrenamiento del modelo, se requiere tokenizar una única frase, por lo que debemos volver a unir los textos superior e inferior, con un detalle adicional... ¡Indicaremos en qué parte se dividen! De esta forma el modelo no sólo será capaz de generar texto de memes, sino que nos indicará en qué parte debemos separarlo. 

De esta forma, utilizamos el caracter `|` para dividirlo. 

Adicionalmente, a cada frase se le va a añadir un token que refleje el id del meme al que pertenece el texto. Así, podremos condicionar el texto generado a partir de un token. Por ejemplo, para el meme asociado a la plantilla 001, el token inicial sería, `<|001|>`.

In [ ]:
def GPT_text(row):
    meme_id = row.meme_id
    text_1 = row.text_1 
    text_2 = row.text_2 
    return '<|'+meme_id+'|>' + text_1.strip() + ' | ' + text_2.strip()

df['GPT_text'] = df.apply(GPT_text, axis = 1)

In [ ]:
df = df.drop_duplicates(subset=['GPT_text', 'meme_id', 'text_1', 'text_2'], keep='last')

In [ ]:
df.meme_id.value_counts()

0004    208152
0003    167777
0012    126011
0007     59834
0011     39244
0008     20869
0006     17993
0013     11293
0005      6086
0010      3049
Name: meme_id, dtype: int64

In [ ]:
df = df.groupby('meme_id').head(20000)

In [ ]:
df.meme_id.value_counts()

0003    20000
0007    20000
0011    20000
0012    20000
0008    20000
0004    20000
0006    17993
0013    11293
0005     6086
0010     3049
Name: meme_id, dtype: int64

# 4. Separación en Entrenamiento y Prueba

Al igual que en cualquier otro modelo, es necesario dividir nuestra muestra en entrenamiento y validación. En este paso tomamos una muestra aletoria de tamaño 10% para ser el test, y el 90% restante será el train. El objetivo de esto es realizar el finetuning sin hacer overfitting y dañar el desempeño del modelo.

In [ ]:
train_text, test_text, train_id, test_id  = train_test_split(df['GPT_text'], df['meme_id'], train_size=.9, random_state=42) # Para hacerlo estratificado

In [ ]:
print(f'Memes en Train: {len(train_text)}\nMemes en Test: {len(test_text)}')

Memes en Train: 142578
Memes en Test: 15843


In [ ]:
model_data_path = os.path.join(drive_path, 'model_data')

train = os.path.join(model_data_path, '20k_sample_train_split.txt')
test = os.path.join(model_data_path, '20k_sample_test_split.txt')

with open(train, 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train_text.values.tolist()))

with open(test, 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(test_text.values.tolist()))

# 5. Entrenamiento de Modelo

En esta sección utilizaremos ya los elementos de texto generados para entrenar el modelo. 

In [ ]:
os.chdir("/content/drive/MyDrive/GPT2/AIMemeGenerator/transformers/examples/")
os.chdir("./language-modeling")

In [ ]:
!python run_clm.py \
--model_type 'datificate/gpt2-small-spanish' \
--model_name_or_path 'datificate/gpt2-small-spanish' \
--train_file '/content/drive/MyDrive/Retrain/AIMemeGenerator/model_data/20k_sample_train_split.txt' \
--do_train \
--validation_file '/content/drive/MyDrive/Retrain/AIMemeGenerator/model_data/20k_sample_test_split.txt' \
--do_eval \
--per_device_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 3 \
--fp16 \
--output_dir='/content/drive/MyDrive/Retrain/AIMemeGenerator/final_model/'

2021-05-03 16:31:54.431290: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
05/03/2021 16:31:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
05/03/2021 16:31:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/content/drive/MyDrive/Retrain/AIMemeGenerator/final_model/, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May03_16-31-56_c0307c08c44c, logging_strategy=I

In [ ]:
# setup imports to use the model
model = TFGPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/GPT2/AIMemeGenerator/saved_model3/', from_pt=True)
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/GPT2/AIMemeGenerator/saved_model3/")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

# 6. Algunos Ejemplos de Generación de texto (sin procesar)

En este punto se prueban diferentes parámetros de la configuración de generación de texto, manipulando el tamaño de la frase. Estos parámetros serán utilizados en el producto final de generación de texto.

In [ ]:
def generate_text(meme_id, n_memes = 1):
  input_ids = tokenizer.encode(f"<|{meme_id}|>", return_tensors='tf')
  generated_text_samples = model.generate(
    input_ids, 
    max_length=40,  
    num_return_sequences=n_memes,
    no_repeat_ngram_size=2,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
  )
  for i, beam in enumerate(generated_text_samples):
    print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=True)))

In [ ]:
generate_text('0004', 3)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: <|0004|>se cree nino y dice que | le importa la madre<0013||asi que eres un joven? | dimeque se siente no saber nada
1: <|0004|>no | le gusta hacer nada en la vida el sabado no tiene otra excusa para que los demas te quieran comer<< |0011|pense  q
2: <|0004|>va al america | se llama yara<<0011||]]le dije que no hariamos nada le dijo que me amaba pero era atea


In [ ]:
generate_text('0007', 3)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0: <|0007|>no se si es una broma | o me esta ignorando<>y a justin bieber le gusta el chico que tiene bigote tu0013|
1: <|0007|>no se si me esta ignorando | o se quiere decir que no quiere dar like a mi facebook<>¡sera feliz!!! no estoy seguro de ti
2: <|0007|>no se si esta es un codigo | o solo piensa que quiere meter a la chica mas hermosa< | dime que se siente ser tan una de las mas
